# Intro
Klik bij iedere cell op 'Run'. Controleer de uitvoer. Notebooks geven een stap-voor-stap uitvoering van code, waardoor gekeken kan worden of het goed verloopt.

In [5]:
import pandas as pd
import requests
import xml.etree.ElementTree as ET

def to_xml(df, file_name=None, root_name='root', row_name='row', name='', index_name='index', do_index=False, use_atts=False):
    root = ET.Element(root_name, name=name)
    tree = ET.ElementTree(root)
    for idx, row in df.iterrows():
        rowelm = ET.SubElement(root, row_name)
        if do_index:
            if use_atts:
                rowelm.set(index_name, str(idx))
            else:
                colelm = ET.SubElement(rowelm, index_name)
                colelm.text = str(idx)
        for hdr, val in row.iteritems():
            hdr = hdr.replace(' ', '_')
            if pd.notnull(val):
                val = str(val)
            else:
                val = ''
            if use_atts:
                rowelm.set(hdr, str(val))
            else:
                colelm = ET.SubElement(rowelm, hdr)
                colelm.text = str(val)
    if file_name is not None:
        tree.write(file_name, xml_declaration=False, encoding='utf8')
    return ET.tostring(root)

# Lees NL vertalingen
Download de Nederlandse vertalingen van LOINC van MLDS: https://mlds.ihtsdotools.org/#/viewReleases/viewRelease/421816
Dit is een Excel bestand. 
Pas de bestandsnaam hieronder (versienummer) aan.

Lees het bestand in.

In [6]:
name = 'LOINC-nlnames-2.74-patch.xlsx'
df = pd.read_excel(name)
df

,LOINC-id,Vertaling
0,80629-9,"1, 2, 2-trimethylpropylmethylfosfonaat [massa/..."
1,54099-7,"1, 2, 2-trimethylpropylmethylfosfonaat [massa/..."
2,17271-8,"1,1,1-trichloorethaan [massa/volume] in bloed"
3,38624-3,"1,1,2-trichloorethaan [massa/volume] in lucht"
4,38334-9,"1,1,2-trichloorethaan [massa/volume] in water"
...,...,...
58717,63103-6,Zwitserse kaas As.IgG [massa/volume] in serum
58718,56399-9,Zwitserse kaas As.IgG4 [massa/volume] in serum
58719,48412-1,Zygomycete sp. Ag [aanwezigheid] in weefsel en...
58720,55198-6,Zygositeit [identificator] in bloed of weefsel...


In [7]:
# @id gets indexed
df.columns = ['id', 'longName']
df[0:20]

,id,longName
0,80629-9,"1, 2, 2-trimethylpropylmethylfosfonaat [massa/..."
1,54099-7,"1, 2, 2-trimethylpropylmethylfosfonaat [massa/..."
2,17271-8,"1,1,1-trichloorethaan [massa/volume] in bloed"
3,38624-3,"1,1,2-trichloorethaan [massa/volume] in lucht"
4,38334-9,"1,1,2-trichloorethaan [massa/volume] in water"
5,38699-5,"1,1-dichloorethaan [massa/volume] in lucht"
6,38735-7,"1,1-dichloorethyleen [massa/volume] in lucht"
7,38292-9,"1,1-dichloorethyleen [massa/volume] in water"
8,74622-2,"1,1-dimethoxy-(9Z)octadeceen [massa/massa] in ..."
9,74623-0,"1,1-dimethoxy-(9Z)octadeceen [mol/massa] in er..."


# Maak XML
De vertalingen worden geëxporteerd naar XML.

In [10]:
xml = to_xml(df, 'nllongnames.xml', root_name='nllongnames', name=name, row_name='concept', use_atts=True)

# Opslaan
Vul target, usernaam, password in. Let op: eerst de cell runnen, dan waarden invullen. Cell niet opnieuw runnen, dat zet de default waarden terug.

In [12]:
import ipywidgets as widgets
targeturi = widgets.Dropdown(
    options=[('Productie', 'https://labterminologie.nl/lab/nllongnames'),
             ('Acceptatie', 'https://terminologie.test-nictiz.nl/lab/nllongnames'),
             ('8877', 'http://localhost:8877/exist/apps/lab/nllongnames')],
    description='Target:',
)
usr = widgets.Text(description='Username')
pwd = widgets.Password(description='Password')
widgets.VBox([
  targeturi, usr, pwd
])

In [17]:
targeturi.value

'https://labterminologie.nl/lab/nllongnames'


# Upload naar de server
Verwachte uitvoer: (HTTP status) 201, dat is 'Created'

In [19]:
resp = requests.put(targeturi.value, data=xml, auth=requests.auth.HTTPBasicAuth(usr.value, pwd.value))
pwd.value = ''
print(resp.status_code, resp.reason)

413 Request Entity Too Large


# Controleer de uitvoer
Onderstaande links voeren naar de prerelease tabel zoals zojuist geüpload.

[productie](http://labterminologie.nl/lab/nllongnames)

[acceptatie](https://terminologie.test-nictiz.nl/lab/nllongnames)

[lokaal 8877](http://localhost:8877/exist/apps/lab-data/data/nllongnames.xml)